In [7]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import layers

### Data Preprocessing

In [13]:
# Define the directories where the train and test data is stored
train_dir = '../data/train'
test_dir = '../data/test'

# Define the image size
img_size = 128

# Function to load images and labels from a directory
def load_data(directory):
    images = []
    labels = []
    for folder in os.listdir(directory):
        if folder == 'damage':
            label = 1
        elif folder == 'no_damage':
            label = 0
        else:
            continue
        for filename in os.listdir(os.path.join(directory, folder)):
            img = cv2.imread(os.path.join(directory, folder, filename), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (img_size, img_size))
            images.append(img)
            labels.append(label)
    images = np.array(images).reshape(-1, img_size, img_size, 1)
    return images, labels

# Load the train and test data
train_images, train_labels = load_data(train_dir)
test_images, test_labels = load_data(test_dir)

# Normalize the pixel values of the images
scaler = MinMaxScaler()
train_images = scaler.fit_transform(train_images.reshape(-1, img_size*img_size))
test_images = scaler.transform(test_images.reshape(-1, img_size*img_size))
train_images = train_images.reshape(-1, img_size, img_size, 1)
test_images = test_images.reshape(-1, img_size, img_size, 1)

# Split the train data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Save the preprocessed images and labels as numpy arrays

val_images = np.array(val_images)
val_labels = np.array(val_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

### Model Creation

In [18]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

Epoch 1/10
250/250 [==============================] - 170s 647ms/step - loss: 0.4611 - accuracy: 0.7804 - val_loss: 0.2396 - val_accuracy: 0.9000
Epoch 2/10
250/250 [==============================] - 153s 612ms/step - loss: 0.2252 - accuracy: 0.9080 - val_loss: 0.1914 - val_accuracy: 0.9235
Epoch 3/10
250/250 [==============================] - 152s 609ms/step - loss: 0.1933 - accuracy: 0.9219 - val_loss: 0.1657 - val_accuracy: 0.9365
Epoch 4/10
250/250 [==============================] - 153s 610ms/step - loss: 0.1542 - accuracy: 0.9373 - val_loss: 0.1316 - val_accuracy: 0.9495
Epoch 5/10
250/250 [==============================] - 155s 621ms/step - loss: 0.1286 - accuracy: 0.9501 - val_loss: 0.1347 - val_accuracy: 0.9490
Epoch 6/10
250/250 [==============================] - 150s 600ms/step - loss: 0.0918 - accuracy: 0.9659 - val_loss: 0.1462 - val_accuracy: 0.9430
Epoch 7/10
250/250 [==============================] - 159s 636ms/step - loss: 0.0695 - accuracy: 0.9772 - val_loss: 0.1319 -

In [9]:
#model.save('./save')
model = keras.models.load_model('../save')

In [28]:
import cv2
import numpy as np
import tensorflow as tf

# Load image
image = cv2.imread('demo3.png')

# Define window size and stride
window_size = (128, 128)
stride = 64

# Generate windows
windows = []
for y in range(0, image.shape[0] - window_size[1] + 1, stride):
    for x in range(0, image.shape[1] - window_size[0] + 1, stride):
        window = image[y:y+window_size[1], x:x+window_size[0]]
        windows.append(window)

# Predict on windows
resized_windows = []
for window in windows:
    resized_window = cv2.resize(window, window_size)
    resized_window = cv2.cvtColor(resized_window, cv2.COLOR_BGR2GRAY)
    # Normalize pixel values of the window
    resized_window = scaler.transform(resized_window.reshape(-1, img_size*img_size)).reshape(img_size, img_size, 1)
    resized_windows.append(resized_window)

resized_windows = np.array(resized_windows)
predictions = model.predict(resized_windows)
predictions = np.squeeze(predictions)

# Define colors for boxes
green_color = (0, 255, 0)
red_color = (0, 0, 255)

# Filter predictions above a certain threshold
threshold = 0.5
damaged_indices = np.where(predictions > threshold)[0]
damaged_count = 0
undamaged_count = 0

# Draw boxes on image
for i, index in enumerate(range(0, len(windows))):
    y = (index // ((image.shape[1] - window_size[0]) // stride)) * stride
    x = (index % ((image.shape[1] - window_size[0]) // stride)) * stride
    color = red_color if i in damaged_indices else green_color
    if i in damaged_indices:
        damaged_count +=1
    else:
        undamaged_count += 1
    cv2.rectangle(image, (x, y), (x+window_size[0], y+window_size[1]), color, thickness=2)

# Save image
cv2.imwrite('your_output_image_path.jpg', image)

print(f'Number of Undamaged Structures: {undamaged_count}')
print(f'Number of Damaged Structures: {damaged_count}')


2/2 [==============================] - 0s 97ms/step
Number of Undamaged Structures: 29
Number of Damaged Structures: 19
